In [ ]:
import os
import json
import numpy as np
from keras import models
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D

from sklearn.model_selection import train_test_split

from skimage.io import imread
from skimage.color import rgb2gray

In [ ]:
img_w = 1280
img_h = 1024
img_layers = 3
n_labels = 9
MASK_COLORS = list(range(n_labels))
kernel = 3

In [ ]:
encoding_layers = [
    Conv2D(64, kernel, padding='same', input_shape=(img_h, img_w, img_layers)),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(128, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(128, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(256, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
]

autoencoder = models.Sequential()
autoencoder.encoding_layers = encoding_layers

for l in autoencoder.encoding_layers:
    autoencoder.add(l)
    print(l.input_shape, l.output_shape, l)

decoding_layers = [
    UpSampling2D(),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(256, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(128, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(128, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(64, (kernel, kernel), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(n_labels, (1, 1), padding='valid'),
    BatchNormalization(),
]
autoencoder.decoding_layers = decoding_layers
for l in autoencoder.decoding_layers:
    autoencoder.add(l)

autoencoder.add(Reshape((n_labels, img_h * img_w)))
autoencoder.add(Permute((2, 1)))
autoencoder.add(Activation('softmax'))

In [ ]:
x_path = "../input/x/"
y_path = "../input/y/"

In [ ]:
def label_map(label):
    x = np.zeros([*label.shape, n_labels])
    for i in range(n_labels):
        x[:,:,i] = label == i
    return x
    

def prep_data():
    labels = np.array([label_map(imread(y_path + file)[:, :, 0]) for file in os.listdir(y_path)])
    labels = labels.reshape((len(img), img_h * img_w, n_labels))
    images = np.array([imread(x_path + file) for file in os.listdir(x_path)]) / 255.

    print('Data prepairing: OK')
    print('\tshapes: {}, {}'.format(img.shape, label.shape))
    print('\ttypes:  {}, {}'.format(img.dtype, label.dtype))
    print('\tmemory: {}, {} MB'.format(img.nbytes / 1048576, label.nbytes / 1048576))
    
    return images, labels

In [ ]:
# Train model or load weights
imgs, labels = prep_data()
X_train, Y_train, X_val, Y_val = train_test_split(imgs, labels, train=0.9)

data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)

seed = 1
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

image_datagen.fit(imgs, augment=True, seed=seed)
mask_datagen.fit(labels, augment=True, seed=seed)

train_images = image_datagen.flow(X_train, y=None, batch_size=32, shuffle=True, sample_weight=None, seed=seed, save_to_dir=None, subset=None)
train_masks = mask_datagen.flow(Y_train, y=None, batch_size=32, shuffle=True, sample_weight=None, seed=seed, save_to_dir=None, subset=None)
val_images = image_datagen.flow(X_val, y=None, batch_size=32, shuffle=True, sample_weight=None, seed=seed, save_to_dir=None, subset=None)
val_masks = mask_datagen.flow(Y_val, y=None, batch_size=32, shuffle=True, sample_weight=None, seed=seed, save_to_dir=None, subset=None)

train_generator = zip(train_images, train_masks)
val_generator = zip(val_images, val_masks)
os.mkdir("results")

In [ ]:
autoencoder.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_acc', min_delta=0.0001,
                           patience=5, verbose=1, mode='auto')
checkpoint = ModelCheckpoint('results/segnet_best.hdf5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')
callbacks = [early_stop, checkpoint]

history = autoencoder.fit_generator(train_generator, steps_per_epochs=100, epochs=500,
                                    validation_generator=val_generator, validation_steps,
                                    verbose=1, callbacks=callbacks)
autoencoder.save_weights('results/model_segnet_trained.hdf5')